## Création du fichier résultat 

L'objectif est d'obtenir deux tableaux (un par feuille) qui regroupent: 
1- une vision globale sur les Objectifs de Développement Durable, 
2- une fiche indicateur dimensionnée sur un filtre dynamique obligatoire.

Import des librairies

In [3]:
import pandas as pd
import openpyxl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print(openpyxl.__version__)

3.1.3


Initialisation du fichier Excel de résultat 

In [6]:
from openpyxl import load_workbook
path_file = '../OUTPUT/TDB_ODD.xlsx'


with pd.ExcelWriter(path_file, mode='a') as writer:
    df.to_excel(writer, sheet_name='DATA', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '../OUTPUT/TDB_ODD.xlsx'

## Tableau bilan 
Elements du tableau: 
    Comparer les quantités d'action, ou l'effort cumulé, sur chaque ODD et par type d'indicateur. 
    Identifier les sujets cibles principaux de développement durable. 
    Identifier les indicateurs les plus représentés (indifféremment de l'ODD concerné). 
    Identifier la répartition du public concerné (sex, âge).
    Identifier la répartition des actions par geolocalisation. 

In [ ]:
# Listes de données 

for each colonne in colonnes :
    



In [ ]:
# Tableau dynamique sur le nombre d'occurrence par ODD
# formule Excel: =